# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating a CDS Contract

In [7]:
valuation_date = Date(20, 8, 2020)
effective_date = Date(21, 8, 2020)
maturity_date = Date(20, 6, 2025)
running_coupon = 0.05
notional = ONE_MILLION
long_protection = True

In [4]:
cds_contract = FinCDS(effective_date, maturity_date, running_coupon, notional, long_protection)

## Build Ibor Curve

In [5]:
settlement_date = Date(24, 8, 2020)

In [8]:
dcType = DayCountTypes.ACT_360
depo1 = FinIborDeposit(settlement_date, "1M", 0.001709, dcType)
depo2 = FinIborDeposit(settlement_date, "2M", 0.002123, dcType)
depo3 = FinIborDeposit(settlement_date, "3M", 0.002469, dcType)
depo4 = FinIborDeposit(settlement_date, "6M", 0.003045, dcType)
depo5 = FinIborDeposit(settlement_date, "12M", 0.004449, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = SwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = FinIborSwap(settlement_date,"2Y", swapType, 0.002155,fixedFreq,dcType)
swap2 = FinIborSwap(settlement_date,"3Y", swapType, 0.002305,fixedFreq,dcType)
swap3 = FinIborSwap(settlement_date,"4Y", swapType, 0.002665,fixedFreq,dcType)
swap4 = FinIborSwap(settlement_date,"5Y", swapType, 0.003290,fixedFreq,dcType)
swap5 = FinIborSwap(settlement_date,"6Y", swapType, 0.004025,fixedFreq,dcType)
swap6 = FinIborSwap(settlement_date,"7Y", swapType, 0.004725,fixedFreq,dcType)
swap7 = FinIborSwap(settlement_date,"8Y", swapType, 0.005430,fixedFreq,dcType)
swap8 = FinIborSwap(settlement_date,"9Y", swapType, 0.006075,fixedFreq,dcType)
swap9 = FinIborSwap(settlement_date,"10Y", swapType, 0.006640,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

libor_curve = FinIborSingleCurve(valuation_date, depos, [], swaps)

Inserting synthetic deposit


# Build a CDS Curve

In [9]:
cdsSpread = 0.01

In [10]:
cds1 = FinCDS(settlement_date, "6M", cdsSpread)
cds2 = FinCDS(settlement_date, "1Y", cdsSpread)
cds3 = FinCDS(settlement_date, "2Y", cdsSpread)
cds4 = FinCDS(settlement_date, "3Y", cdsSpread)
cds5 = FinCDS(settlement_date, "4Y", cdsSpread)
cds6 = FinCDS(settlement_date, "5Y", cdsSpread)
cds7 = FinCDS(settlement_date, "7Y", cdsSpread)
cds8 = FinCDS(settlement_date, "10Y", cdsSpread)

In [11]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [12]:
recovery_rate = 0.40

In [14]:
issuer_curve = FinCDSCurve(valuation_date, cdss, libor_curve, recovery_rate)

In [15]:
print(issuer_curve)

OBJECT TYPE: FinCDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5808219,  0.9901921
 1.0849315,  0.9817974
 2.0849315,  0.9653478
 3.0849315,  0.9491785
 4.0876712,  0.9332392
 5.0876712,  0.9176136
 7.0876712,  0.8871536
10.0904110,  0.8433231


In [16]:
cds_contract.print_flows(issuer_curve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
MON 21 SEP 2020   0.252778     12638.89     0.999848     0.998513     12618.18
MON 21 DEC 2020   0.252778     12638.89     0.999079     0.994298     12555.25
MON 22 MAR 2021   0.252778     12638.89     0.998005     0.990100     12488.80
MON 21 JUN 2021   0.252778     12638.89     0.996528     0.985940     12417.92
MON 20 SEP 2021   0.252778     12638.89     0.995677     0.981797     12355.18
MON 20 DEC 2021   0.252778     12638.89     0.996304     0.977670     12310.99
MON 21 MAR 2022   0.252778     12638.89     0.996610     0.973560     12263.01
MON 20 JUN 2022   0.252778     12638.89     0.996071     0.969468     12204.85
TUE 20 SEP 2022   0.255556     12777.78     0.995501     0.965348     12279.50
TUE 20 DEC 2022   0.252778     12638.89     0.994878     0.961291     12087.41
MON 20 MAR 2023   0.250000     12500.00     0.994249     0.957295     11897.38
TUE 20 JUN 2023   0.255556     12777.78     0.993573   

# Valuation

In [17]:
spd = cds_contract.par_spread(settlement_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00001 bp


In [18]:
cds_contract.value_fast_approx(settlement_date, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [19]:
v = cds_contract.value(settlement_date, issuer_curve, recovery_rate)

In [20]:
full_pv = v['full_pv'] 
clean_pv = v['clean_pv']

In [21]:
print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -195423.92
CLEAN VALUE   -187090.59


In [22]:
cleanp = cds_contract.clean_price(settlement_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.709065


In [23]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 60


In [29]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8333.333333333334


In [30]:
prot_pv = cds_contract.protection_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 46771.65011225943


In [31]:
premPV = cds_contract.premium_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 242192.6470217891


In [32]:
cds_contract.risky_pv01(settlement_date, issuer_curve)

{'full_rpv01': 4.843852940435782, 'clean_rpv01': 4.677186273769115}

## Risk Measures

In [33]:
cds_contract.credit_dv01(settlement_date, issuer_curve, recovery_rate)

542.5032361687918

In [34]:
cds_contract.interest_dv01(settlement_date, issuer_curve, recovery_rate)

46.52493245183723

Copyright (c) 2020 Dominic O'Kane